In [364]:
def is_white(pixel_val):
    """tuple with (r, g, b) values"""
    return (pixel_val[0] + pixel_val[1] + pixel_val[2]) > 250*3 or pixel_val[1] > 250 # allow a small noise for white

def inside_image(coords):
    global img
    global width
    global height
    return (coords[0] >= 0 and coords[1] >= 0) and (coords[0] < width and coords[1] < height)

def extend_dir(coords, direction):
    global img
    global width
    global height
    temp_coords = coords
    for magnitude in range(max(width, height)+2):
        if not inside_image(temp_coords):
            return magnitude
        if is_white(img[temp_coords]):
            return magnitude
        temp_coords = (temp_coords[0] + direction[0], temp_coords[1] + direction[1])
    return width

def border_distance(coords):
    return min(extend_dir(coords, (-1, 0)), extend_dir(coords, (1, 0)), extend_dir(coords, (0, 1)), extend_dir(coords, (0, -1)))

def max_distance_pixel():
    global width
    global height
    max_border_distance = 0
    candidate_pixel = (1, 1)
    for i in range(0, width, 9):
        for j in range(0, height, 9):
            if i%60 == 0 and j == 0:
                print("yolo")
            pixel_distance = border_distance((i, j))
            if pixel_distance > max_border_distance:
                max_border_distance = pixel_distance
                candidate_pixel = (i, j)
    print(max_border_distance, "lol")
    return candidate_pixel

In [365]:
def expand_strategy(pixel, strategy):
    up = 0
    down = 0
    left = 0
    right = 0
    expanded = True
    while expanded:
        expanded = False
        
        for move in strategy:
            if move == 0: # 0 is left
                if expand_left_right(pixel[1]-up, pixel[1]+down, pixel[0]-left-1):
                    expanded = True
                    left += 1
            
            elif move == 1: # 1 is right
                if expand_left_right(pixel[1]-up, pixel[1]+down, pixel[0]+right+1):
                    expanded = True
                    right += 1
            
            elif move == 2: # 2 is up
                if expand_up_down(pixel[0]-left, pixel[0]+right, pixel[1]-up-1):
                    expanded = True
                    up += 1
                    
            else: # 3 is down
                if expand_up_down(pixel[0]-left, pixel[0]+right, pixel[1]+down+1):
                    expanded = True
                    down += 1
            
    return (left, right, up, down)

def expand_rectangle(pixel):
    best_area = 0
    best_coords = (0, 0, 0, 0)
    
    strategies = [(0, 0, 1, 1, 2, 2, 3, 3), (3, 3, 2, 2, 0, 0, 1, 1), (0, 2, 1, 3),\
                  (2, 0, 3, 1), (0, 0, 1, 1, 2, 3), (3, 3, 2, 2, 0, 1)]
    
    for strategy in strategies:
        new_coords = expand_strategy(pixel, strategy)
        new_area = (new_coords[1]+new_coords[0])*(new_coords[2]+new_coords[3])
        if new_area > best_area:
            best_area = new_area
            best_coords = new_coords
    
    return best_coords
        
def expand_up_down(left, right, line):
    global img
    for i in range(left, right+1):
        if not inside_image((i, line)) or is_white(img[(i, line)]):
            return False # cannot expand
    return True

def expand_left_right(up, down, line):
    global img
    for i in range(up, down+1):
        if not inside_image((line, i)) or is_white(img[(line, i)]):
            return False # cannot expand
    return True

In [366]:
def fill_white(ul, lr): # fill in rectangle from upper-left to lower-right
    global img
    for col in range(ul[0], lr[0]+1):
        for row in range(ul[1], lr[1]+1):
            img[col, row] = (0, 255, 0) # make everything white!

In [367]:
def find_blank(num_points=40):
    boxes = []
    for num in range(num_points):
        point = max_distance_pixel()
        (left, right, up, down) = expand_rectangle(point)
        fill_white((point[0]-left, point[1]-up), (point[0]+right, point[1]+down))
        boxes.append(((point[0]-left, point[1]-up), (point[0]+right, point[1]+down)))
        #fill_white((point[0]-10, point[1]-10), (point[0]+10, point[1]+10))
    return boxes

In [371]:
im = Image.open("empty_w2.jpg")
im1 = im.filter(ImageFilter.FIND_EDGES)
im2 = im1.filter(ImageFilter.MaxFilter(size=7))
rgb_im1 = im2.convert('RGB')
rgb_im1.show()
global img
global width
global height
width, height = rgb_im1.size
img = rgb_im1.load()
boxes = find_blank()

yolo
yolo
yolo
yolo


KeyboardInterrupt: 

In [ ]:
rgb_im1.show()